In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train=pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train[train["Age"].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q


In [5]:
train_x=train.drop(["PassengerId","Survived","Name","Ticket","Cabin"],axis=1)
train_y=train["Survived"]

In [6]:
train_x[train_x["Embarked"]=='C'].count()[0]


168

In [7]:
train[(train["Embarked"] == 'C') & (train["Survived"] == 1)].count()[0]
#0.5535714285714286

93

In [8]:
93/168

0.5535714285714286

In [9]:
train_x[train_x["Embarked"]=='Q'].count()[0]


77

In [10]:
train[(train["Embarked"] == 'Q') & (train["Survived"] == 1)].count()[0]
#0.38961038961038963

30

In [11]:
30/77

0.38961038961038963

In [12]:
train_x[train_x["Embarked"]=='S'].count()[0]

644

In [13]:
train[(train["Embarked"] == 'S') & (train["Survived"] == 1)].count()[0]
#0.33695652173913043

217

In [14]:
217/644

0.33695652173913043

In [15]:
train_x["Embarked"]=train_x["Embarked"].fillna("C")

In [16]:
train_x['Sex'] =train_x['Sex'].replace({'male': 0, 'female': 1})
train_x['Embarked'] =train_x['Embarked'].replace({'S': 0, 'C': 1,'Q':2})
train_x["Age"]=train_x["Age"].fillna(train_x["Age"].median())

In [17]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [18]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,max_depth=5,random_state=17)
rf.fit(train_x,train_y)

RandomForestClassifier(max_depth=5, random_state=17)

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_val_score)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

results = cross_val_score(rf, train_x, train_y, cv=skf)
print("CV accuracy score: {:.2f}%".format(results.mean() * 100))

CV accuracy score: 83.05%


In [20]:
x_test=test.drop(["Name","PassengerId","Ticket","Cabin"],axis=1)
x_test["Sex"]=x_test["Sex"].replace({"male":0,"female":1})
x_test['Embarked'] =x_test['Embarked'].replace({'S': 0, 'C': 1,'Q':2})
x_test["Age"]=x_test["Age"].fillna(x_test["Age"].median())

In [21]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


In [22]:
x_test[x_test["Fare"].isnull()].head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
152,3,0,60.5,0,0,NaN,0


In [23]:
x_test["Fare"]=x_test["Fare"].fillna(7.8792)

In [24]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


In [25]:
predictions=rf.predict(x_test)
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
